# Conversational AI Demo

Interactive demonstration of the voice assistant capabilities.

In [ ]:
import sys
sys.path.insert(0, '..')

from src.nlu.pipeline import NLUPipeline
from src.dialog.state_tracker import DialogStateTracker
from src.dialog.policy import DialogPolicy
from src.dialog.response_generator import ResponseGenerator

## 1. NLU Pipeline Demo

In [ ]:
nlu = NLUPipeline(use_transformer_ner=False, enable_sentiment=False)

test_queries = [
    "What's the weather in San Francisco?",
    "Set a timer for 5 minutes",
    "Play some jazz music",
    "Remind me to call mom tomorrow at 3pm",
]

for query in test_queries:
    result = nlu.process(query)
    print(f"Query: {query}")
    print(f"  Intent: {result.intent.intent} ({result.intent.confidence:.2f})")
    print(f"  Entities: {[e.to_dict() for e in result.entities.entities]}")
    print()

## 2. Dialog Management Demo

In [ ]:
dialog = DialogStateTracker()
policy = DialogPolicy()
generator = ResponseGenerator()

session_id = "demo-session"

conversation = [
    "Hello",
    "Set a timer",
    "5 minutes",
]

for user_input in conversation:
    nlu_result = nlu.process(user_input)
    
    state = dialog.update(
        session_id=session_id,
        user_utterance=user_input,
        intent=nlu_result.intent.intent,
        entities=[e.to_dict() for e in nlu_result.entities.entities],
    )
    
    action = policy.select_action(state)
    response = generator.generate(action)
    
    print(f"User: {user_input}")
    print(f"Bot: {response}")
    print(f"  [Turn {state.turn_count}, Intent: {state.active_intent}]")
    print()

## 3. ASR Demo

In [ ]:
import numpy as np
from src.asr.whisper_asr import WhisperASR, WhisperConfig

# Note: This uses mock mode without actual Whisper model
config = WhisperConfig(model_size="tiny")
asr = WhisperASR(config)

# Create test audio (1 second of silence)
audio = np.zeros(16000, dtype=np.float32)

result = asr.transcribe(audio)
print(f"Transcription: {result.text}")
print(f"Language: {result.language}")
print(f"Duration: {result.duration}s")
print(f"Processing time: {result.processing_time*1000:.1f}ms")

## 4. TTS Demo

In [ ]:
from src.tts.synthesizer import NeuralTTS, TTSConfig

# Note: This uses mock mode without actual TTS model
config = TTSConfig()
tts = NeuralTTS(config)

result = tts.synthesize("Hello, how can I help you today?")

print(f"Audio samples: {len(result.audio)}")
print(f"Duration: {result.duration:.2f}s")
print(f"Sample rate: {result.sample_rate}Hz")
print(f"Processing time: {result.processing_time*1000:.1f}ms")